In [1]:
import os
import gradio as gr
import uuid
import urllib.request
from fpdf import FPDF
from langchain_ollama import OllamaLLM
import zipfile
import shutil

llm = OllamaLLM(model="gemma3:4b")

# ✅ Ensure Font Exists
FONT_PATH = "DejaVuSans.ttf"
FONT_ZIP_URL = "https://downloads.sourceforge.net/project/dejavu/dejavu/2.37/dejavu-fonts-ttf-2.37.zip"

def ensure_font():
    if not os.path.exists(FONT_PATH):
        print("⏬ Downloading DejaVuSans.ttf...")
        font_zip_path = "dejavu-fonts.zip"
        urllib.request.urlretrieve(FONT_ZIP_URL, font_zip_path)

        with zipfile.ZipFile(font_zip_path, 'r') as zip_ref:
            zip_ref.extractall("dejavu-extracted")

        for root, dirs, files in os.walk("dejavu-extracted"):
            for file in files:
                if file == "DejaVuSans.ttf":
                    shutil.copy(os.path.join(root, file), FONT_PATH)
                    print("✅ Font extracted and ready.")
                    break

ensure_font()  # Run at startup


# ✅ Improved LLM Wrapper
def call_llm(system_prompt, user_prompt):
    prompt = f"{system_prompt}\n\n{user_prompt}"
    response = llm.invoke(prompt)

    # Ensure response is text
    if hasattr(response, "content"):
        return response.content.strip()
    elif isinstance(response, str):
        return response.strip()
    else:
        return "⚠️ Error: Unexpected response format"


# ✅ Improved Pitch Generator
def generate_startup_pitch(theme):
    try:
        # Ensure font is available
        font_path = "DejaVuSans.ttf"
        if not os.path.exists(font_path):
            print("⏬ Downloading DejaVuSans.ttf...")
            font_zip_url = "https://downloads.sourceforge.net/project/dejavu/dejavu/2.37/dejavu-fonts-ttf-2.37.zip"
            font_zip_path = "dejavu-fonts.zip"

            urllib.request.urlretrieve(font_zip_url, font_zip_path)

            with zipfile.ZipFile(font_zip_path, 'r') as zip_ref:
                zip_ref.extractall("dejavu-extracted")

            for root, dirs, files in os.walk("dejavu-extracted"):
                for file in files:
                    if file == "DejaVuSans.ttf":
                        ttf_path = os.path.join(root, file)
                        shutil.copy(ttf_path, font_path)
                        print("✅ Font extracted and ready.")
                        break

        idea_prompt = f"Generate an innovative startup idea in the field of {theme}."
        tagline_prompt = "Generate a short, catchy tagline for the startup."
        pitch_prompt = "Write a concise pitch deck covering Problem, Solution, Market, Team, and Business Model."

        idea = call_llm("You are an innovation strategist.", idea_prompt)
        tagline = call_llm("You are a branding expert.", tagline_prompt)
        pitch = call_llm("You are a startup mentor writing a pitch deck.", pitch_prompt)

        print(f"DEBUG: Idea={idea}, Tagline={tagline}, Pitch={pitch}")  # Debugging output

        if not idea or not tagline or not pitch:
            raise ValueError("⚠️ Error: Missing response from call_llm()!")

        filename = os.path.join(os.getcwd(), f"startup_pitch_{uuid.uuid4().hex[:8]}.pdf")
        print(f"DEBUG: Saving PDF to {filename}")

        pdf = FPDF()
        pdf.add_page()
        pdf.add_font("DejaVu", "", font_path, uni=True)
        pdf.set_font("DejaVu", size=12)
        pdf.multi_cell(0, 10, f"Startup Idea:\n{idea}\n\nTagline:\n{tagline}\n\nPitch Deck:\n{pitch}")
        pdf.output(filename)

        return idea, tagline, pitch, filename

    except Exception as e:
        print(f"⚠️ Error in generate_startup_pitch: {e}")
        return f"⚠️ Error: {e}", "", "", None


# ✅ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 AI Startup Pitch Generator (with PDF Export)")
    theme_input = gr.Textbox(label="Enter a theme or industry", placeholder="e.g., mental health, fintech, climate tech")
    generate_button = gr.Button("Generate Pitch")

    idea_output = gr.Textbox(label="Startup Idea")
    tagline_output = gr.Textbox(label="Tagline")
    pitch_output = gr.Textbox(label="Pitch Deck Summary", lines=10)
    pdf_output = gr.File(label="Download Pitch as PDF")

    # ✅ Ensure the correct file path is returned
    def wrapper(theme):
        idea, tagline, pitch, pdf_path = generate_startup_pitch(theme)
        return idea, tagline, pitch, pdf_path if os.path.isfile(pdf_path) else None

    generate_button.click(fn=wrapper, inputs=theme_input, outputs=[idea_output, tagline_output, pitch_output, pdf_output])

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
